
# List of Packages to Download

```python
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub
!pip install -qU langchain-openai
!pip install --upgrade --quiet  rank_bm25 > /dev/null
!pip install faiss-cpu
!pip install faiss-gpu
```
**Make sure to have an OPEN AI Api key available**
______________________________________________________________________________
 

# 1. Load Data


In [1]:
from langchain_community.document_loaders import WebBaseLoader


In [7]:
import os

#os.environ["OPENAI_API_KEY"] = ""

In [2]:
#https://en.wikipedia.org/wiki/LeBron_James
# "https://www.reddit.com/r/NBATalk/comments/1co5rhc/if_lebron_is_in_the_goat_debate_so_should_kobe/"
loader =  WebBaseLoader(["https://bleacherreport.com/articles/10117950-michael-jordan-narrowly-tops-lebron-james-as-nba-goat-kobe-bryant-3rd-in-player-poll","https://www.reddit.com/r/NBATalk/comments/1co5rhc/if_lebron_is_in_the_goat_debate_so_should_kobe/" ])
data = loader.load()
data

[Document(page_content="Michael Jordan Narrowly Tops LeBron James as NBA GOAT; Kobe Bryant 3rd in Player Poll | News, Scores, Highlights, Stats, and Rumors | Bleacher ReportXBleacher ReportNBANEWSSCORESTEAMSRUMORSHIGHLIGHTSFANTASYDRAFTBR Sports on MaxBleacher ReportNBAWORLD FOOTBALLNFLAEWWWEMMABOXINGSPORTSSCORESAccountNBANEWSSCORESTEAMSRUMORSHIGHLIGHTSFANTASYDRAFTBR Sports on MaxFacebook LogoCopy Link IconHome>NBA>Los Angeles LakersMichael Jordan Narrowly Tops LeBron James as NBA GOAT; Kobe Bryant 3rd in Player PollTyler Conway@@jtylerconwayFeatured Columnist IVApril 22, 2024 CommentsComment Bubble IconJoe Murphy/NBAE via Getty ImagesAccording to NBA players, the GOAT debate consists of only two players and everyone else is in a distant rearview mirror.Michael Jordan edged out LeBron James as the greatest player in NBA history, according to a poll of 133 players conducted by The Athletic. Jordan received 45.9 percent of the vote, while James came in at 42.1 percent.Kobe Bryant was a di

# 2. Make Ensemble of Retrievers

In [3]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

### Retriever 1: BM25 retriever

In [4]:
bm25_retriever = BM25Retriever.from_documents(data) #first retriever

bm25_retriever.k = 2

### Retriever 2: FAISS retriever

In [8]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
faiss_vectorstore = FAISS.from_documents(data, embedding) #vectorstore

faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2}) #second retriever

### Combined Retriver 1 + Retriver 2

In [9]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)


# 3. Make a RAG pipeline


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [11]:
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")


In [12]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=ensemble_retriever,
        chain_type_kwargs={"prompt": prompt}
    )

In [13]:
question = "Who is the GOAT based upon people's opinions"
result = qa_chain({"query": question })

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

("Based on people's opinions, Michael Jordan is narrowly considered the GOAT "
 '(Greatest of All Time) in NBA history, edging out LeBron James according to '
 'a poll of 133 NBA players. Jordan received 45.9 percent of the vote, while '
 'James received 42.1 percent. Kobe Bryant came in third with 9.8 percent of '
 'the vote.')
